In [1]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create a table to store data from multiple sources
cursor.execute('''
CREATE TABLE IF NOT EXISTS calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER NOT NULL,
    month INTEGER NOT NULL,
    day INTEGER NOT NULL,
    week_number INTEGER NOT NULL,
    quarter INTEGER NOT NULL,
    day_of_week TEXT NOT NULL,
    is_weekend BOOLEAN NOT NULL,
    full_date DATETIME NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')



# Commit changes and close the connection
conn.commit()
conn.close()

print("Database and table created successfully.")

Database and table created successfully.


In [2]:
from datetime import datetime, timedelta

# Generate records from 1900 to 2030
start_date = datetime(1900, 1, 1)
end_date = datetime(2030, 12, 31)
current_date = start_date

records = []
while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    week_number = current_date.isocalendar()[1]
    quarter = (month - 1) // 3 + 1
    day_of_week = current_date.strftime('%A')
    is_weekend = day_of_week in ['Saturday', 'Sunday']
    full_date = current_date.strftime('%Y-%m-%d')
    
    records.append((year, month, day, week_number, quarter, day_of_week, is_weekend, full_date))
    current_date += timedelta(days=1)

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Insert records into the calendar table
cursor.executemany('''
INSERT INTO calendar (year, month, day, week_number, quarter, day_of_week, is_weekend, full_date)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', records)

# Commit changes
conn.commit()

# Close the connection after the operation
conn.close()

print(f"{len(records)} records inserted into the calendar table.")

47847 records inserted into the calendar table.


In [3]:
import pandas as pd

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')

# Read the calendar table into a pandas DataFrame
calendar_df = pd.read_sql_query("SELECT * FROM calendar", conn)

# Close the connection
conn.close()

# Display the DataFrame
calendar_df.head()

,id,year,month,day,week_number,quarter,day_of_week,is_weekend,full_date,timestamp
0,1,1900,1,1,1,1,Monday,0,1900-01-01,2025-04-19 16:35:22
1,2,1900,1,2,1,1,Tuesday,0,1900-01-02,2025-04-19 16:35:22
2,3,1900,1,3,1,1,Wednesday,0,1900-01-03,2025-04-19 16:35:22
3,4,1900,1,4,1,1,Thursday,0,1900-01-04,2025-04-19 16:35:22
4,5,1900,1,5,1,1,Friday,0,1900-01-05,2025-04-19 16:35:22


In [ ]:
import fastf1

# Enable the cache for fastf1
import os

# Ensure the cache directory exists
cache_dir = 'fastf1_cache'
os.makedirs(cache_dir, exist_ok=True)

# Enable the cache for fastf1
fastf1.Cache.enable_cache(cache_dir)

# Initialize an empty list to store F1 calendar data
f1_calendar_data = []

# Loop through the years from 2000 to 2025
for year in range(2000, 2026):
    # Load the F1 schedule for the current year
    schedule = fastf1.get_event_schedule(year)
    
    # Extract relevant data for the table
    for _, event in schedule.iterrows():
        race_name = event['EventName']
        country = event['Country']
        circuit = event['Location']
        event_year = event['EventDate'].year
        date = event['EventDate'].strftime('%Y-%m-%d')
        f1_calendar_data.append((race_name, country, circuit, event_year, date))

# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the f1_calendar table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_calendar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    race_name TEXT NOT NULL,
    country TEXT NOT NULL,
    circuit TEXT NOT NULL,
    year INTEGER NOT NULL,
    date TEXT NOT NULL
)
''')

# Insert F1 calendar data into the table
cursor.executemany('''
INSERT INTO fone_calendar (race_name, country, circuit, year, date)
VALUES (?, ?, ?, ?, ?)
''', f1_calendar_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("F1 calendar data from 2000 to 2025 inserted into the database successfully.")


Added Australian Grand Prix in Australia on 2000-03-12
Added Brazilian Grand Prix in Brazil on 2000-03-26
Added San Marino Grand Prix in Italy on 2000-04-09
Added British Grand Prix in UK on 2000-04-23
Added Spanish Grand Prix in Spain on 2000-05-07
Added European Grand Prix in Germany on 2000-05-21
Added Monaco Grand Prix in Monaco on 2000-06-04
Added Canadian Grand Prix in Canada on 2000-06-18
Added French Grand Prix in France on 2000-07-02
Added Austrian Grand Prix in Austria on 2000-07-16
Added German Grand Prix in Germany on 2000-07-30
Added Hungarian Grand Prix in Hungary on 2000-08-13
Added Belgian Grand Prix in Belgium on 2000-08-27
Added Italian Grand Prix in Italy on 2000-09-10
Added United States Grand Prix in USA on 2000-09-24
Added Japanese Grand Prix in Japan on 2000-10-08
Added Malaysian Grand Prix in Malaysia on 2000-10-22
Added Australian Grand Prix in Australia on 2001-03-04
Added Malaysian Grand Prix in Malaysia on 2001-03-18
Added Brazilian Grand Prix in Brazil on 2

In [ ]:
# Reopen the database connection
conn = sqlite3.connect('planet_fone.db')
cursor = conn.cursor()

# Create the fone_geography table
cursor.execute('''
CREATE TABLE IF NOT EXISTS fone_geography (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    circuit TEXT NOT NULL,
    city TEXT NOT NULL,
    country TEXT NOT NULL,
    continent TEXT NOT NULL,
    latitude REAL NOT NULL,
    longitude REAL NOT NULL
)
''')

# Define a mapping of circuits to their geographic information
# This is a sample dataset; you should replace it with accurate data
geography_data = [
    ('Yas Island', 'Abu Dhabi', 'United Arab Emirates', 'Asia', 24.4672, 54.6031),
    ('Silverstone', 'Silverstone', 'United Kingdom', 'Europe', 52.0733, -1.0144),
    ('Monza', 'Monza', 'Italy', 'Europe', 45.6206, 9.2811),
    # Add more circuits here...
]

# Insert unique circuit data into the fone_geography table
cursor.executemany('''
INSERT INTO fone_geography (circuit, city, country, continent, latitude, longitude)
VALUES (?, ?, ?, ?, ?, ?)
''', geography_data)

# Add a geo_id column to the fone_calendar table and establish the foreign key relationship
cursor.execute('ALTER TABLE fone_calendar ADD COLUMN geo_id INTEGER')
cursor.execute('''
UPDATE fone_calendar
SET geo_id = (
    SELECT id FROM fone_geography
    WHERE fone_geography.circuit = fone_calendar.circuit
)
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("fone_geography table created and linked to fone_calendar successfully.")